In [39]:
import bs4
from urllib.request import urlopen

In [40]:
url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930'
url_float

'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930'

In [41]:
source = urlopen(url_float).read()
soup = bs4.BeautifulSoup(source,'lxml')

In [42]:
#//*[@id="cTB11"]/tbody/tr[7]/td

In [43]:
soup.find(id = 'cTB11').find_all('tr')[6].td.text

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5,969,782,550주 / 78.79%\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [44]:
tmp = soup.find(id = 'cTB11').find_all('tr')[6].td.text
tmp = tmp.replace('\r','')
tmp = tmp.replace('\n','')
tmp = tmp.replace('\t','')
tmp

'5,969,782,550주 / 78.79%'

In [45]:
import re

In [46]:
tmp = re.split('/', tmp)
tmp

['5,969,782,550주 ', ' 78.79%']

In [47]:
tmp[1]

' 78.79%'

In [48]:
tmp[0]

'5,969,782,550주 '

In [49]:
outstanding = tmp[0].replace(',', '')
outstanding = outstanding.replace('주','')
outstanding = outstanding.replace(' ','')
outstanding

'5969782550'

In [50]:
floating = tmp[1].replace(' ','')
floating = floating.replace('%','')
floating

'78.79'

In [51]:
outstanding = int(outstanding)
outstanding

5969782550

In [52]:
type(outstanding)

int

In [53]:
floating = float(floating)
floating

78.79

In [54]:
type(floating)

float

//*[@id="cTB11"]/tbody/tr[7]/td

In [55]:
#구성종목 기본정보
def stock_info(stock_cd):
    url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + stock_cd
    source = urlopen(url_float).read()
    soup = bs4.BeautifulSoup(source, 'lxml')
    
    tmp = soup.find(id = 'cTB11').find_all('tr')[6].td.text
    tmp = tmp.replace('\r','')
    tmp = tmp.replace('\n','')
    tmp = tmp.replace('\t','')
    
    tmp = re.split('/',tmp)
    
    outstanding = tmp[0].replace(',','')
    outstanding = outstanding.replace('주','')
    outstanding = outstanding.replace(' ','')
    outstanding = int(outstanding)
    
    floating = tmp[1].replace(' ','')
    floating = floating.replace('%','')
    floating = float(floating)
    
    name = soup.find(id='pArea').find('div').find('div').find('tr').find('td').find('span').text
    
    k10_outstanding[stock_cd] = outstanding
    k10_floating[stock_cd] = floating
    k10_name[stock_cd] = name

In [56]:
k10_component = ['005930', '000660', '068270', '207940', '005380', '005490', '051910', '028260', '035420', '012330']

In [57]:
k10_outstanding = dict() #딕셔너리 초기화
k10_floating = dict()
k10_name = dict()

for stock_cd in k10_component:    #종목 하나씩 돌리면서 stock info 함수 호출
    stock_info(stock_cd)

In [58]:
k10_outstanding     #상장 주식수

{'005930': 5969782550,
 '000660': 728002365,
 '068270': 127953489,
 '207940': 66165000,
 '005380': 213668187,
 '005490': 87186835,
 '051910': 70592343,
 '028260': 189690043,
 '035420': 164813395,
 '012330': 97343863}

In [59]:
k10_floating       #유동비율

{'005930': 78.79,
 '000660': 73.89,
 '068270': 66.22,
 '207940': 24.75,
 '005380': 65.77,
 '005490': 79.6,
 '051910': 64.28,
 '028260': 52.08,
 '035420': 78.77,
 '012330': 67.12}

In [60]:
#/html/body/table[1]/tbody/tr[3]/td[1]/span

In [61]:
import pandas as pd
import datetime as dt

In [62]:
def date_format(d):
    d = str(d).replace('-','.')
    
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy,mm,dd)
    return this_date

In [63]:
def historical_stock_naver(stock_cd, start_date = '', end_date = '', page_n = 1, last_page = 0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
        
    naver_stock = 'https://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_stock).read()
    source = bs4.BeautifulSoup(source, 'lxml')
    
    dates = source.find_all('span', class_ = 'tah p10 gray03') #날짜
    prices = source.find_all('td', class_ = 'num') #종가
    
    for n in range(len(dates)):
        
        if len(dates) > 0:
            #날짜처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                #start_date 와 end_date 사이에서 데이터 저장
                
                this_close = prices[n*6].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
                
                #딕셔너리 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:
                return historical_prices
            
        if last_page == 0:
            last_page = source.find_all('table')[1].find_all('td', class_ = 'pgRR').find('a')['href']
            last_page = last_page.split('&')[1]
            last_page = last_page.split('=')[1]
            last_page = float(last_page)
            
    if page_n < last_page:
        page_n = page_n + 1
        historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices

In [64]:
k10_historical_prices = dict()

for stock_cd in k10_component:
    
    historical_prices = dict()
    start_date = '2017-1-1'
    end_date = '2017-12-31'
    historical_stock_naver(stock_cd, start_date, end_date)
    
    k10_historical_prices[stock_cd] = historical_prices

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?